In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
import cactas as C

In [3]:
import tensorflow as tf
gpu_available = tf.test.is_gpu_available()
gpu_available

2023-09-26 16:51:21.157462: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2023-09-26 16:51:24.037316: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 16:51:24.040013: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-09-26 16:51:24.392190: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:47:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-09-26 16:51:24.394745: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:4e:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 3

True

In [4]:
DATAPATH='/raid/mpsych/CACTAS/DATA/ESUS'

In [5]:
images, labels = C.Helper.load_data(DATAPATH)

In [6]:
X_train, y_train, X_test, y_test = C.Helper.split_patients(images, labels)

In [7]:
X_train, y_train, X_test, y_test = C.Helper.normalization(DATAPATH, X_train, y_train, X_test, y_test)

In [8]:
X_train, y_train, X_test, y_test = C.Helper.extract_slices(X_train, y_train, X_test, y_test)

(13068, 512, 512, 1) (13068, 512, 512, 1) (3698, 512, 512, 1) (3698, 512, 512, 1)


In [9]:
X_train, y_train, X_test, y_test = C.Helper.filter_slices(X_train, y_train, X_test, y_test)

(577, 512, 512, 1) (577, 512, 512, 1) (137, 512, 512, 1) (137, 512, 512, 1)


In [10]:
train_gen = C.Helper.augment(X_train, y_train)

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [11]:
model = C.Helper.create_unet(X_train[0].shape)

2023-09-26 16:53:57.408581: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:47:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-09-26 16:53:57.411300: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:4e:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-09-26 16:53:57.421030: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0, 1
2023-09-26 16:53:57.424051: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:47:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-09-26 16:53:57.427765: I tensorflow/core

In [12]:
model, history = C.Helper.train_unet(train_gen, X_train, y_train, X_test, y_test, model, epochs=500)

2023-09-26 16:53:58.727660: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-09-26 16:53:58.748752: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2245800000 Hz


Epoch 1/500


2023-09-26 16:53:59.961496: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-09-26 16:54:00.950433: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201
2023-09-26 16:54:02.123228: W tensorflow/stream_executor/gpu/asm_compiler.cc:191] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.0
2023-09-26 16:54:02.123248: W tensorflow/stream_executor/gpu/asm_compiler.cc:194] Used ptxas at ptxas
2023-09-26 16:54:02.123606: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Unimplemented: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-09-26 16:54:02.221509: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-09-26 16:54:03.366270: I tensorflow/stream_exec

19/19 [==============================] - 48s 1s/step - loss: 0.2155 - iou: 9.8070e-05 - iou_thresholded: 0.0023 - val_loss: 0.0611 - val_iou: 4.6778e-06 - val_iou_thresholded: 7.9668e-04
Epoch 2/500
19/19 [==============================] - 18s 963ms/step - loss: 0.0324 - iou: 4.9781e-05 - iou_thresholded: 0.0022 - val_loss: 0.0243 - val_iou: 2.7428e-05 - val_iou_thresholded: 7.9668e-04
Epoch 3/500
19/19 [==============================] - 18s 967ms/step - loss: 0.0166 - iou: 1.2762e-04 - iou_thresholded: 0.0017 - val_loss: 0.0111 - val_iou: 7.5474e-05 - val_iou_thresholded: 7.9668e-04
Epoch 4/500
19/19 [==============================] - 18s 967ms/step - loss: 0.0076 - iou: 0.0012 - iou_thresholded: 0.0073 - val_loss: 0.0035 - val_iou: 2.1382e-04 - val_iou_thresholded: 7.9668e-04
Epoch 5/500
19/19 [==============================] - 18s 963ms/step - loss: 0.0032 - iou: 4.2365e-04 - iou_thresholded: 0.0014 - val_loss: 0.0027 - val_iou: 3.9271e-04 - val_iou_thresholded: 7.9668e-04
Epoch 6/5

19/19 [==============================] - 18s 965ms/step - loss: 6.0565e-04 - iou: 0.0656 - iou_thresholded: 0.0019 - val_loss: 0.0011 - val_iou: 0.0682 - val_iou_thresholded: 7.9668e-04
Epoch 43/500
19/19 [==============================] - 18s 960ms/step - loss: 5.9093e-04 - iou: 0.0729 - iou_thresholded: 0.0066 - val_loss: 8.2915e-04 - val_iou: 0.0913 - val_iou_thresholded: 7.9668e-04
Epoch 44/500
19/19 [==============================] - 18s 966ms/step - loss: 5.8418e-04 - iou: 0.0745 - iou_thresholded: 0.0020 - val_loss: 8.0008e-04 - val_iou: 0.1033 - val_iou_thresholded: 7.9668e-04
Epoch 45/500
19/19 [==============================] - 18s 963ms/step - loss: 6.2458e-04 - iou: 0.0720 - iou_thresholded: 0.0038 - val_loss: 0.0010 - val_iou: 0.0772 - val_iou_thresholded: 7.9668e-04
Epoch 46/500
19/19 [==============================] - 18s 962ms/step - loss: 6.0187e-04 - iou: 0.0747 - iou_thresholded: 0.0018 - val_loss: 0.0014 - val_iou: 0.0579 - val_iou_thresholded: 9.8441e-04
Epoch 47/5

...

Epoch 250/500
19/19 [==============================] - 18s 960ms/step - loss: 3.9366e-05 - iou: 0.8473 - iou_thresholded: 0.9109 - val_loss: 0.0021 - val_iou: 0.4533 - val_iou_thresholded: 0.4669
Epoch 251/500
19/19 [==============================] - 18s 967ms/step - loss: 3.5396e-05 - iou: 0.8322 - iou_thresholded: 0.8823 - val_loss: 0.0027 - val_iou: 0.4327 - val_iou_thresholded: 0.4454
Epoch 252/500
19/19 [==============================] - 18s 961ms/step - loss: 3.8201e-05 - iou: 0.8531 - iou_thresholded: 0.9096 - val_loss: 0.0022 - val_iou: 0.4402 - val_iou_thresholded: 0.4616
Epoch 253/500
19/19 [==============================] - 18s 962ms/step - loss: 8.4355e-05 - iou: 0.7628 - iou_thresholded: 0.8166 - val_loss: 0.0015 - val_iou: 0.4450 - val_iou_thresholded: 0.4638
Epoch 254/500
19/19 [==============================] - 18s 964ms/step - loss: 5.7820e-05 - iou: 0.8051 - iou_thresholded: 0.8695 - val_loss: 0.0016 - val_iou: 0.4451 - val_iou_thresholded: 0.4596
Epoch 255/500
19/19 

Epoch 292/500
19/19 [==============================] - 18s 961ms/step - loss: 1.9794e-05 - iou: 0.9166 - iou_thresholded: 0.9623 - val_loss: 0.0036 - val_iou: 0.4289 - val_iou_thresholded: 0.4342
Epoch 293/500
19/19 [==============================] - 18s 963ms/step - loss: 1.8162e-05 - iou: 0.9196 - iou_thresholded: 0.9609 - val_loss: 0.0037 - val_iou: 0.4237 - val_iou_thresholded: 0.4293
Epoch 294/500
19/19 [==============================] - 18s 962ms/step - loss: 2.2993e-05 - iou: 0.9155 - iou_thresholded: 0.9529 - val_loss: 0.0031 - val_iou: 0.4448 - val_iou_thresholded: 0.4515
Epoch 295/500
19/19 [==============================] - 18s 961ms/step - loss: 2.1544e-05 - iou: 0.9163 - iou_thresholded: 0.9572 - val_loss: 0.0034 - val_iou: 0.4294 - val_iou_thresholded: 0.4355
Epoch 296/500
19/19 [==============================] - 18s 964ms/step - loss: 1.8018e-05 - iou: 0.9291 - iou_thresholded: 0.9669 - val_loss: 0.0036 - val_iou: 0.4332 - val_iou_thresholded: 0.4378
Epoch 297/500
19/19 

Epoch 334/500
19/19 [==============================] - 18s 963ms/step - loss: 4.6496e-05 - iou: 0.8193 - iou_thresholded: 0.8920 - val_loss: 0.0018 - val_iou: 0.4097 - val_iou_thresholded: 0.4267
Epoch 335/500
19/19 [==============================] - 18s 960ms/step - loss: 4.1425e-05 - iou: 0.8254 - iou_thresholded: 0.8987 - val_loss: 0.0019 - val_iou: 0.4121 - val_iou_thresholded: 0.4280
Epoch 336/500
19/19 [==============================] - 18s 966ms/step - loss: 4.4556e-05 - iou: 0.8316 - iou_thresholded: 0.8955 - val_loss: 0.0020 - val_iou: 0.4097 - val_iou_thresholded: 0.4225
Epoch 337/500
19/19 [==============================] - 18s 959ms/step - loss: 3.6801e-05 - iou: 0.8533 - iou_thresholded: 0.9189 - val_loss: 0.0021 - val_iou: 0.4102 - val_iou_thresholded: 0.4216
Epoch 338/500
19/19 [==============================] - 18s 967ms/step - loss: 3.1489e-05 - iou: 0.8727 - iou_thresholded: 0.9312 - val_loss: 0.0022 - val_iou: 0.4155 - val_iou_thresholded: 0.4257
Epoch 339/500
19/19 

Epoch 376/500
19/19 [==============================] - 18s 964ms/step - loss: 1.5483e-05 - iou: 0.9483 - iou_thresholded: 0.9753 - val_loss: 0.0031 - val_iou: 0.4199 - val_iou_thresholded: 0.4252
Epoch 377/500
19/19 [==============================] - 18s 962ms/step - loss: 1.2007e-05 - iou: 0.9508 - iou_thresholded: 0.9746 - val_loss: 0.0032 - val_iou: 0.4155 - val_iou_thresholded: 0.4205
Epoch 378/500
19/19 [==============================] - 18s 963ms/step - loss: 1.2689e-05 - iou: 0.9505 - iou_thresholded: 0.9739 - val_loss: 0.0032 - val_iou: 0.4153 - val_iou_thresholded: 0.4183
Epoch 379/500
19/19 [==============================] - 18s 962ms/step - loss: 4.0072e-05 - iou: 0.9102 - iou_thresholded: 0.9405 - val_loss: 0.0029 - val_iou: 0.4018 - val_iou_thresholded: 0.4101
Epoch 380/500
19/19 [==============================] - 18s 960ms/step - loss: 2.2875e-05 - iou: 0.9215 - iou_thresholded: 0.9554 - val_loss: 0.0026 - val_iou: 0.4242 - val_iou_thresholded: 0.4323
Epoch 381/500
 8/19 

KeyboardInterrupt: 

In [ ]:
C.Helper.visualize_graph(history)

In [ ]:
y_pred = C.Helper.prediction(X_test, model)

In [ ]:
C.Helper.visualize_result(X_test, y_test, y_pred)

In [ ]:
C.Helper.evaluate(X_test, y_test, model)